In [ ]:
# Creating a function to clean car details seperate it

import pandas as pd
import ast


def new_car_details_cleaning(file_path,output_path):
  df = pd.read_excel(file_path)

# Specify the columns containing dictionary string
  dict_columns = ['new_car_detail', 'new_car_overview', 'new_car_feature', 'new_car_specs']
  
# Create an Excel writer to save each dictionary as a separate sheet
  #output_path = r'C:\Users\loges\Desktop\python\sample projects\GUVI\MLapp\data\delhi.xlsx'
  with pd.ExcelWriter(output_path) as writer:
    for col in dict_columns:
        # Check if the column exists in the DataFrame
        if col in df.columns:
            # Convert each cell in the column from a dictionary string to an actual dictionary
            try:
                column_data = df[col].dropna().apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else {})
            except ValueError as e:
                print(f"Error parsing column {col}: {e}")
                continue  # Skip this column if there's an error

            # Normalize each dictionary into a DataFrame and combine into a single DataFrame for the sheet
            table_df = pd.json_normalize(column_data)

            # Debug: print the head of the table to confirm data processing
            print(f"Processed data for column {col}:\n", table_df.head())

            # Write each table to a separate sheet
            table_df.to_excel(writer, sheet_name=col, index=False)
        else:
            print(f"Column '{col}' does not exist in the DataFrame.")

# Output path to download the file
  print("Data written to:", output_path)


In [ ]:
# creating a function to clean car_overveiw file

import pandas as pd
import ast

def clean_and_expand_new_car_overview(file_path, sheet_name='new_car_overview', data_column='top'):
    """
    This function reads an Excel file, cleans the data, expands dictionary-like data into separate columns,
    and saves the cleaned data back to the same Excel file.

    Parameters:
    - file_path (str): The path to the Excel file.
    - sheet_name (str): The name of the sheet to process. Default is 'new_car_overview'.
    - data_column (str): The column containing dictionary-like data. Default is 'top'.
    """

    # Load the Excel file
    df = pd.read_excel(file_path, sheet_name=sheet_name)

    # Function to remove outer brackets from the data
    def remove_outer_brackets(value):
        if isinstance(value, str) and value.startswith('[') and value.endswith(']'):
            return value[1:-1]  # Remove first and last character (outer brackets)
        return value

    # Apply the function to remove outer brackets from the specified column
    df[data_column] = df[data_column].apply(remove_outer_brackets)

    # Function to parse each cell's dictionary-like data and convert it into a dictionary of columns
    def expand_data(cell_value):
        try:
            # Convert the string representation of dictionary list to actual Python list
            data_list = ast.literal_eval(cell_value)
            # Create a dictionary to hold the expanded data for each row
            expanded_row = {}
            # Extract 'key', 'value', and 'icon' for each item and set them in the expanded row
            for item in data_list:
                key = item.get('key')
                value = item.get('value')
                icon = item.get('icon')
                # Create columns for 'value' and 'icon' associated with each key
                expanded_row[f'{key}'] = value
                expanded_row[f'{key} Icon'] = icon
            return expanded_row
        except (ValueError, SyntaxError):
            return {}

    # Apply the function and expand each cell into separate columns
    expanded_data = df[data_column].apply(expand_data).apply(pd.Series)

    # Concatenate the expanded columns with the original DataFrame (excluding the original data column)
    df_cleaned = pd.concat([df.drop(columns=[data_column]), expanded_data], axis=1)

    # Write the cleaned DataFrame back to the same file, replacing the original sheet
    with pd.ExcelWriter(file_path, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
        df_cleaned.to_excel(writer, sheet_name=sheet_name, index=False)

    print("The data has been expanded into columns and saved to the same file, with 'key' as column names and 'value' and 'icon' as data.")

# Example usage:


In [ ]:
# creating a function to clean car_feature file

import pandas as pd
import ast

def clean_and_expand_new_car_feature(file_path, sheet_name='new_car_feature'):
    """
    This function reads an Excel file, cleans the data, expands list of dictionaries into separate columns,
    and saves the cleaned data back to the same Excel file.

    Parameters:
    - file_path (str): The path to the Excel file.
    - sheet_name (str): The name of the sheet to process. Default is 'new_car_feature'.
    """

    # Load the Excel file
    df = pd.read_excel(file_path, sheet_name=sheet_name)

    # Function to remove outer brackets from the data
    def remove_outer_brackets(value):
        if isinstance(value, str) and value.startswith('[') and value.endswith(']'):
            return value[1:-1]  # Remove first and last character (outer brackets)
        return value

    # Apply the function to remove outer brackets from the 'top' and 'data' columns
    df['top'] = df['top'].apply(remove_outer_brackets)
    df['data'] = df['data'].apply(remove_outer_brackets)

    # Function to expand each list of dictionaries within a cell
    def expand_dict_list(cell_value):
        try:
            # Convert cell value to list of dictionaries
            data_list = ast.literal_eval(cell_value)
            # Extract each dictionary's 'value' into a new list for each row
            values = [item['value'] for item in data_list if 'value' in item]
            return values
        except (ValueError, SyntaxError):
            return []

    # Expand the 'top' and 'data' columns separately
    top_expanded = df['top'].apply(expand_dict_list).apply(pd.Series)
    data_expanded = df['data'].apply(expand_dict_list).apply(pd.Series)

    # Rename columns to avoid duplication in the final table
    top_expanded.columns = [f"top_{i+1}" for i in range(top_expanded.shape[1])]
    data_expanded.columns = [f"data_{i+1}" for i in range(data_expanded.shape[1])]

    # Concatenate expanded data with original DataFrame (excluding 'top' and 'data' columns)
    df_cleaned = pd.concat([df.drop(columns=['top', 'data']), top_expanded, data_expanded], axis=1)

    # Save the updated data back to the same sheet in the original Excel file
    with pd.ExcelWriter(file_path, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
        df_cleaned.to_excel(writer, sheet_name=sheet_name, index=False)

    print("The data has been expanded and saved to the same sheet in the original file.")



In [ ]:
# creating a function to clean car_specs sheet

import pandas as pd
import ast

def clean_new_car_specs(file_path):
    """
    Cleans the 'new_car_specs' sheet in the given Excel file.
    - Converts JSON-like strings into structured data.
    - Flattens key-value pairs into separate columns.
    - Saves the cleaned data back to the same file.

    :param file_path: Path to the Excel file.
    """
    # Load the Excel file
    xls = pd.ExcelFile(file_path)
    
    # Load the 'new_car_specs' sheet
    df_specs = pd.read_excel(xls, sheet_name='new_car_specs')

    # Function to safely convert stringified lists/dicts to Python objects
    def safe_eval(val):
        try:
            return ast.literal_eval(val) if isinstance(val, str) else val
        except (SyntaxError, ValueError):
            return None

    # Apply transformation to 'top' and 'data' columns
    if 'top' in df_specs.columns:
        df_specs['top'] = df_specs['top'].apply(safe_eval)
    
    if 'data' in df_specs.columns:
        df_specs['data'] = df_specs['data'].apply(safe_eval)

    # Flatten 'top' column into separate columns if it contains data
    if df_specs['top'].notna().any():
        top_df = df_specs['top'].apply(lambda x: {d['key']: d['value'] for d in x} if isinstance(x, list) else {}).apply(pd.Series)
        df_specs = pd.concat([df_specs, top_df], axis=1)

    # Drop unneeded columns
    df_specs.drop(columns=['heading', 'commonIcon', 'top'], inplace=True, errors='ignore')

    # Save cleaned data back to the same file
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        df_specs.to_excel(writer, sheet_name='new_car_specs', index=False)

    print("Data cleaning complete. File updated successfully.")




Cleaning Delhi cars

In [ ]:
file_path = r'C:\Users\loges\Desktop\python\sample projects\GUVI\MLapp\data\delhi.xlsx'

clean_and_expand_new_car_overview(file_path)

new_car_details_cleaning(r'c:\Users\loges\Downloads\delhi_cars (3).xlsx',r'C:\Users\loges\Desktop\python\sample projects\GUVI\MLapp\data\delhi.xlsx')

clean_and_expand_new_car_feature(file_path)

clean_new_car_specs(file_path)

Cleaning kolkata cars

In [ ]:
file_path = r"C:\Users\loges\Desktop\python\sample projects\GUVI\MLapp\data\kolkata_cars.xlsx"

new_car_details_cleaning(file_path,output_path=file_path)

clean_and_expand_new_car_overview(file_path)

clean_and_expand_new_car_feature(file_path)

clean_new_car_specs(file_path)

Cleaning Jaipur cars

In [ ]:
file_path = r"C:\Users\loges\Desktop\python\sample projects\GUVI\MLapp\data\jaipur_cars.xlsx"

new_car_details_cleaning(file_path,output_path=file_path)

clean_and_expand_new_car_overview(file_path)

clean_and_expand_new_car_feature(file_path)

clean_new_car_specs(file_path)

Cleaning chennai cars

In [ ]:
file_path = r"C:\Users\loges\Desktop\python\sample projects\GUVI\MLapp\data\chennai_cars.xlsx"

new_car_details_cleaning(file_path,output_path=file_path)

clean_and_expand_new_car_overview(file_path)

clean_and_expand_new_car_feature(file_path)

clean_new_car_specs(file_path)

Cleaning Hydrabad cars

In [ ]:
file_path = r"C:\Users\loges\Desktop\python\sample projects\GUVI\MLapp\data\hyderabad_cars.xlsx"

new_car_details_cleaning(file_path,output_path=file_path)

clean_and_expand_new_car_overview(file_path)

clean_and_expand_new_car_feature(file_path)

clean_new_car_specs(file_path)

Cleaning Bangalore cars

In [ ]:
file_path = r"C:\Users\loges\Desktop\python\sample projects\GUVI\MLapp\data\bangalore_cars.xlsx"

new_car_details_cleaning(file_path,output_path=file_path)

clean_and_expand_new_car_overview(file_path)

clean_and_expand_new_car_feature(file_path)

clean_new_car_specs(file_path)

In [10]:


import pandas as pd


def concat_features(file_path):
   

# Path to your Excel file
 excel_file = file_path

# Read all sheets into a dictionary of DataFrames
 all_sheets = pd.read_excel(excel_file, sheet_name=None)

# Combine all sheets horizontally (side by side)
 combined_df = pd.concat(all_sheets.values(), axis=1)

# Save the combined DataFrame to a new Excel file (optional)
 combined_df.to_excel(file_path, index=False)

# Display the combined DataFrame
 print(combined_df)
    
    # Display the combined DataFrame
   


In [ ]:
concat_features(r"C:\Users\loges\Desktop\python\sample projects\GUVI\MLapp\data\delhi.xlsx")


      it      ft         bt      km transmission  ownerNo      owner  \
0      0  Diesel        SUV  10,000    Automatic        1  1st Owner   
1      0  Petrol        SUV  57,437       Manual        2  2nd Owner   
2      0  Petrol        SUV   8,000    Automatic        1  1st Owner   
3      0  Petrol  Hatchback  28,151       Manual        2  2nd Owner   
4      0  Petrol        SUV  60,000       Manual        1  1st Owner   
...   ..     ...        ...     ...          ...      ...        ...   
1480   0  Petrol      Sedan  60,000    Automatic        1  1st Owner   
1481   0  Petrol        SUV  20,000       Manual        1  1st Owner   
1482   0  Petrol      Sedan  60,000       Manual        1  1st Owner   
1483   0  Diesel        SUV  38,000       Manual        1  1st Owner   
1484   0  Diesel        SUV   7,000    Automatic        0  0th Owner   

                oem              model  modelYear  ...  \
0               Kia         Kia Seltos       2022  ...   
1           Hyundai

In [ ]:
concat_features(r"C:\Users\loges\Desktop\python\sample projects\GUVI\MLapp\data\chennai_cars.xlsx")
concat_features(r"C:\Users\loges\Desktop\python\sample projects\GUVI\MLapp\data\bangalore_cars.xlsx")
concat_features(r"C:\Users\loges\Desktop\python\sample projects\GUVI\MLapp\data\hyderabad_cars.xlsx")
concat_features(r"C:\Users\loges\Desktop\python\sample projects\GUVI\MLapp\data\kolkata_cars.xlsx")
concat_features(r"C:\Users\loges\Desktop\python\sample projects\GUVI\MLapp\data\jaipur_cars.xlsx")

In [28]:
import pandas as pd
df1 = pd.read_excel(r"C:\Users\loges\Desktop\python\sample projects\GUVI\MLapp\data\delhi.xlsx")
df2 = pd.read_excel(r"C:\Users\loges\Desktop\python\sample projects\GUVI\MLapp\data\bangalore_cars.xlsx")
df3 = pd.read_excel(r"C:\Users\loges\Desktop\python\sample projects\GUVI\MLapp\data\chennai_cars.xlsx")
df4 = pd.read_excel(r"C:\Users\loges\Desktop\python\sample projects\GUVI\MLapp\data\hyderabad_cars.xlsx")
df5 = pd.read_excel(r"C:\Users\loges\Desktop\python\sample projects\GUVI\MLapp\data\kolkata_cars.xlsx")
df6 = pd.read_excel(r"C:\Users\loges\Desktop\python\sample projects\GUVI\MLapp\data\jaipur_cars.xlsx")


final_data=pd.concat([df1,df2,df3,df4,df5,df6],axis=0)
final_data.to_excel(r"C:\Users\loges\Desktop\python\sample projects\GUVI\MLapp\features_with_additional_data.xlsx")


In [29]:
df=final_data.drop(['priceActual', 'priceSaving', 'priceFixedText', 'heading', 'Unnamed: 1',
                  'trendingText.desc','bottomData','data','Registration Year Icon',
                 'Insurance Validity Icon','Fuel Type Icon','Seats Icon',
                  'Kms Driven Icon' ,'Ownership Icon','Engine Displacement Icon',
                  'Transmission Icon','Year of Manufacture Icon','RTO Icon','heading.1','commonIcon','Engine','Seats.1','owner','trendingText.imgUrl',
                   'trendingText.heading' ]
                  ,axis=1)


In [30]:



# Remove all non-numeric characters while keeping numbers and decimals
# Assuming df is your DataFrame
columns_to_clean = ['price', 'Registration Year', 'Seats', 'Kms Driven', 
                    'Engine Displacement', 'Mileage', 'Max Power', 'Torque','Wheel Size']

# Remove non-numeric characters while keeping numbers and decimals
df[columns_to_clean] = df[columns_to_clean].replace(r'[^0-9.]', '', regex=True)

# Convert to numeric type
df[columns_to_clean] = df[columns_to_clean].apply(pd.to_numeric, errors='coerce')

print(df.head())

   it      ft         bt      km transmission  ownerNo            oem  \
0   0  Diesel        SUV  10,000    Automatic        1            Kia   
1   0  Petrol        SUV  57,437       Manual        2        Hyundai   
2   0  Petrol        SUV   8,000    Automatic        1  Mercedes-Benz   
3   0  Petrol  Hatchback  28,151       Manual        2         Maruti   
4   0  Petrol        SUV  60,000       Manual        1        Hyundai   

               model  modelYear  centralVariantId  ...   top_4  \
0         Kia Seltos       2022              7121  ...  Heater   
1      Hyundai Creta       2016              1449  ...  Heater   
2  Mercedes-Benz GLC       2020              6837  ...  Heater   
3       Maruti Swift       2018             10403  ...  Heater   
4      Hyundai Creta       2017              5362  ...  Heater   

                    top_5                                       top_6  \
0  Adjustable Head Lights                            Fog Lights Front   
1  Adjustable Head

In [31]:
# seperating column by dtype for imputational process

numeric_df = df.select_dtypes(include=['number'])
non_numeric_df = df.select_dtypes(exclude=['number'])

nu_col = numeric_df.columns
non_col = non_numeric_df.columns

In [60]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

# Apply label encoding to categorical columns
df= df.apply(lambda x : le.fit_transform(x) if x.dtypes == 'object' else x)


In [ ]:
import matplotlib.pyplot as plt

null=df.isnull()

plt.figure(figsize=(10,20))

null.sum().plot(kind='barh')

In [62]:
columns = df.columns

In [17]:
print(numeric_df)

      it  ownerNo  modelYear  centralVariantId  price  Registration Year  \
0      0        1       2022              7121  19.00             2022.0   
1      0        2       2016              1449   7.62             2016.0   
2      0        1       2020              6837  56.00             2020.0   
3      0        2       2018             10403   5.23             2019.0   
4      0        1       2017              5362   7.65             2017.0   
...   ..      ...        ...               ...    ...                ...   
1115   0        1       2011               438   1.08             2011.0   
1116   0        1       2007              3917   1.05             2007.0   
1117   0        1       2022              8350   8.50             2022.0   
1118   0        1       2009              3904   1.00             2009.0   
1119   0        1       2018              3962   5.00             2018.0   

      Seats  Kms Driven  Engine Displacement  Year of Manufacture  Mileage  \
0       5

In [14]:
from sklearn.experimental import enable_iterative_imputer  # Enable IterativeImputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge

imputer = IterativeImputer(estimator=BayesianRidge(), max_iter=10, random_state=42)

# Fit and transform the data
imputed_data = imputer.fit_transform(df)

# Convert back to DataFrame
df = pd.DataFrame(imputed_data, columns=df.columns)

ValueError: could not convert string to float: 'Diesel'

In [32]:
#run if the current trying algorithm is not supported the iterative imputation

from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)

numeric_df= imputer.fit_transform(numeric_df)

numeric_df = pd.DataFrame(numeric_df, columns =nu_col  )

imputer1 = SimpleImputer(strategy = "constant")
non_numeric_df = imputer1.fit_transform(non_numeric_df)

non_numeric_df = pd.DataFrame(non_numeric_df, columns = non_col )

df = pd.concat([numeric_df, non_numeric_df], axis=1)

In [66]:
df.to_excel(r'C:\Users\loges\Desktop\python\sample projects\GUVI\MLapp\car_features.xlsx')

In [21]:
import pandas as pd

df = pd.read_excel(r'C:\Users\loges\Desktop\python\sample projects\GUVI\MLapp\car_features.xlsx')

# Replace 0 values in the 'price' column with the mean of the non-zero values
df['price'] = df['price'].replace(0, df['price'].mod())

df.to_excel(r'C:\Users\loges\Desktop\python\sample projects\GUVI\MLapp\car_features.xlsx', index=False)

TypeError: Series.mod() missing 1 required positional argument: 'other'

In [ ]:
import pandas as pd

# Load the data
#df = pd.read_excel(r'C:\Users\loges\Desktop\python\sample projects\GUVI\MLapp\car_features.xlsx')

# Calculate the mean of non-zero values in the 'price' column
mean_price = df[df['price'] != 0]['price'].mean()

# Replace 0 values in the 'price' column with the mean of the non-zero values
df['price'] = df['price'].replace(0, mean_price)


df['price']


In [41]:
df.to_excel(r'C:\Users\loges\Desktop\python\sample projects\GUVI\MLapp\car_features_without_impute.xlsx', index=False)

In [40]:
df

,ft,bt,km,oem,transmission,ownerNo,modelYear,Engine Displacement,Seats,price
0,Diesel,SUV,"10,000",Kia,Automatic,1.0,2022.0,1493.0,5.0,19.00
1,Petrol,SUV,"57,437",Hyundai,Manual,2.0,2016.0,1591.0,5.0,7.62
2,Petrol,SUV,"8,000",Mercedes-Benz,Automatic,1.0,2020.0,1991.0,5.0,56.00
3,Petrol,Hatchback,"28,151",Maruti,Manual,2.0,2018.0,1197.0,5.0,5.23
4,Petrol,SUV,"60,000",Hyundai,Manual,1.0,2017.0,1591.0,5.0,7.65
...,...,...,...,...,...,...,...,...,...,...
8364,Petrol,Hatchback,"1,20,000",Chevrolet,Manual,1.0,2011.0,1199.0,5.0,1.08
8365,Petrol,Hatchback,"70,000",Maruti,Manual,1.0,2007.0,796.0,5.0,1.05
8366,Petrol,SUV,"20,000",Renault,Manual,1.0,2022.0,999.0,5.0,8.50
8367,Petrol,Hatchback,"40,000",Maruti,Manual,1.0,2009.0,796.0,5.0,1.00


In [39]:
df=df[['ft','bt','km','oem','transmission','ownerNo','modelYear','Engine Displacement',"Seats","price"]]

In [ ]:
import pandas as pd

df = pd.read_excel(r'car_features.xlsx')

In [ ]:
df=df[['ft','bt','km','oem','transmission','ownerNo','modelYear','Engine Displacement',"Seats","price"]]

,ft,bt,km,oem,transmission,ownerNo,modelYear,Engine Displacement,Seats,price
0,Diesel,SUV,"10,000",Kia,Automatic,1.0,2022.0,1493.0,5.0,19.00
1,Petrol,SUV,"57,437",Hyundai,Manual,2.0,2016.0,1591.0,5.0,7.62
2,Petrol,SUV,"8,000",Mercedes-Benz,Automatic,1.0,2020.0,1991.0,5.0,56.00
3,Petrol,Hatchback,"28,151",Maruti,Manual,2.0,2018.0,1197.0,5.0,5.23
4,Petrol,SUV,"60,000",Hyundai,Manual,1.0,2017.0,1591.0,5.0,7.65
...,...,...,...,...,...,...,...,...,...,...
8364,Petrol,Hatchback,"1,20,000",Chevrolet,Manual,1.0,2011.0,1199.0,5.0,1.08
8365,Petrol,Hatchback,"70,000",Maruti,Manual,1.0,2007.0,796.0,5.0,1.05
8366,Petrol,SUV,"20,000",Renault,Manual,1.0,2022.0,999.0,5.0,8.50
8367,Petrol,Hatchback,"40,000",Maruti,Manual,1.0,2009.0,796.0,5.0,1.00
